# Warsztaty z uczenia maszynowego A: problemy rozpoznawania mowy
## Automatyczne Rozpoznawania Mowy (ASR) w pigułce - tutorial Kaldi
###### 2020.05.15
---
###Marcin Sikora 
###(marcin.sikora at amu.edu.pl)
---

**Zaliczenia**:
1. Obecności
1. Zadania z 2-części warsztatów (2020-04-17, 2020-05-15)
1. Ocena końcowa to średnia ocena z 2-części


Termin zgłaszania zadań - do końca semestru.

Skala ocen:
- 3: zadanie 1.
- 4: zadanie 1, niedokończone zadanie 2. 
- 5: zadanie 1, 2.

**Agenda**:
1. Przygotowanie środowiska
1. Wprowadzenie teoretyczne
1. Trening modelu minilibrispeech
1. Zadania na zaliczenie

**Przydatne linki/literatura**:

- https://kaldi-asr.org/doc/kaldi_for_dummies.html
- https://kaldi-asr.org/doc/data_prep.html (jak przygotować dane do treningu/testów)
- https://github.com/tvarog/kaldi-minilibrispeech-tutorial (ten tutorial + zmodyfikowana recepta Kaldi
- Dan Jurafsky, James H. Martin, Speech and Language Processing (3rd ed. draft), https://web.stanford.edu/~jurafsky/slp3/
- Kaldi documentation, http://kaldi-asr.org/doc/ 
- Sanjeev Khudanpur, Dan Povey, Jan Trmal, Building Speech Recognition Systems with the Kaldi Toolkit, https://www.clsp.jhu.edu/wp-content/uploads/2016/06/Building-Speech-Recognition-Systems-with-the-Kaldi-Toolkit.pdf




![Kaldi](https://kaldi-asr.org/doc/KaldiTextAndLogoSmall.png)
##Czym jest Kaldi?
Kaldi to zestaw narzędzi open source napisany w C++, bashu, perlu i pythonie służący do budowania systemów rozpoznawania mowy i przetwarzania sygnałów, dostępny bezpłatnie na licencji Apache v2.0. Kaldi teoretycznie działa na Windowsie, ale w praktyce - nie polecam. Kaldi najlepiej czuje się na Linuksie, działa też po niewielkich modyfikacjach na OS X.

Za pomocą Kaldi możemy:
- zamienić sygnał mowy na cechy,
- wytrenować i/lub zaadaptować modele akustyczne i językowe,
- użyć wytrenowanych modeli do zdekodowania sygnału mowy,
- i wiele innych.

Na dzisiejszych zajęciach:
- skonfigurujesz środowisko
- skompilujesz Kaldi wraz z innymi niezbędnymi narzędziami,
- przygotujesz dane do wytrenowania bardzo prostego modelu rozpoznawania mowy
- wytrenujesz modele statystyczne - HMM-GMM oraz model neuronowy (chain)


Na początku sprawdźmy czy runtime do którego połączyliśmy się jest instancją GPU:


In [1]:
!nvidia-smi
!tar xaf drive/My\ Drive/Colab\ Notebooks/kaldi.tar.gz

Thu May 14 21:39:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Jeśli output polecenia nvidia-smi to:

```
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

```
należy zmienić instancję na instancję GPU (`Menu Runtime -> Change runtime type -> GPU`). Uruchom polecenie raz jeszcze żeby potwierdzić, że jesteś na instancji GPU.

Następnie sklonujmy repozytorium Kaldi:

In [3]:
!git clone https://github.com/kaldi-asr/kaldi.git

Cloning into 'kaldi'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 111059 (delta 11), reused 13 (delta 3), pack-reused 111029
Receiving objects: 100% (111059/111059), 118.98 MiB | 33.67 MiB/s, done.
Resolving deltas: 100% (86105/86105), done.


Aby zainstalować Kaldi, należy:
- zainstalować/skompilować zewnętrzne narzędzia (`kaldi/tools/INSTALL`)
- skompilować kod Kaldi (`kaldi/src/INSTALL`)

Zacznijmy od narzędzi:

In [0]:
!cat kaldi/tools/INSTALL

Według instrukcji skrypt `kaldi/tools/extras/check_dependencies.sh` wylistuje pakiety które należy doinstalować, sprawdźmy więc:

In [9]:
!kaldi/tools/extras/check_dependencies.sh

kaldi/tools/extras/check_dependencies.sh: WARNING python 2.7 is not the default python. We fixed this by adding a correct symlink more prominently on the path.
 ... If you really want to use python 3.6.9 as default, add an empty file /content/python/.use_default_python and run this script again.
kaldi/tools/extras/check_dependencies.sh: all OK.


Jak miło - dostaliśmy gotowe polecenia do instalacji wszystkich potrzebnych pakietów. Instalujemy:
- `automake, autoconf` - narzędzia do automatycznego tworzenia plików Makefile
- `sox` - bardzo użyteczne narzędzie do analizy i przetwarzania plików audio
- `libtool` - narzędzie do linkowania binarek i bibliotek
- `subversion` - inaczej SVN (poprzednik gita) 
- `mkl` - zestaw bibliotek do algebry numerycznej od Intela
- Od siebie dodam jeszcze pakiet `flac` który będzie nam potrzebny do rozpakowania nagrań mowy w dalszej części tutoriala oraz `tree` do wyświetlania ładnie sformatowanych drzewek katalogów.

Warning o złej wersji pythona można zignorować.

In [8]:
!sudo apt-get install automake autoconf sox libtool subversion flac tree
!kaldi/tools/extras/install_mkl.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  autotools-dev file libapr1 libaprutil1 libmagic-mgc libmagic1
  libopencore-amrnb0 libopencore-amrwb0 libserf-1-1 libsigsegv2
  libsox-fmt-alsa libsox-fmt-base libsox3 libsvn1 m4
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc gettext libsox-fmt-all
  libtool-doc gcj-jdk m4-doc db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  autoconf automake autotools-dev file flac libapr1 libaprutil1 libmagic-mgc
  libmagic1 libopencore-amrnb0 libopencore-amrwb0 libserf-1-1 libsigsegv2
  libsox-fmt-alsa libsox-fmt-base libsox3 libsvn1 libtool m4 sox subversion
  tree
0 upgraded, 22 newly installed, 0 to remove and 54 not upgraded.
Need to get 4,479 kB of archives.
After this operation, 22.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe am

Po zainstalowaniu pakietów z repozytoriów, resztę należy niestety skompilować. Trochę to potrwa. Zainstalujemy:
- OpenFST - biblioteka do obsługi automatów skończenie stanowych
- Sctk - narzędzie do obliczania wyników ewaluacji

`-j2` spowoduje uruchomienie kompilacji w dwóch procesach.

In [6]:
!cd kaldi/tools && make -j2

extras/check_dependencies.sh
tar --no-same-owner -xzf sph2pipe_v2.5.tar.gz
if [ -d "" ]; then \
	cp -p "/sctk-2.4.10-20151007-1312Z.tar.bz2" .; \
else \
	wget -T 10 https://www.openslr.org/resources/4/sctk-2.4.10-20151007-1312Z.tar.bz2; \
fi
--2020-05-14 19:26:10--  https://www.openslr.org/resources/4/sctk-2.4.10-20151007-1312Z.tar.bz2
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... extras/check_dependencies.sh: all OK.
if [ -d "" ]; then \
	cp -p "/openfst-1.6.7.tar.gz" .; \
else \
	wget -T 10 -t 1 http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.7.tar.gz || \
	wget -T 10 -t 3 -c https://www.openslr.org/resources/2/openfst-1.6.7.tar.gz; \
fi
--2020-05-14 19:26:10--  http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.7.tar.gz
Resolving www.openfst.org (www.openfst.org)... 206.196.111.47
Connecting to www.openfst.org (www.openfst.org)|206.196.111.47|:80... connected.
HTTP request sen

Narzędzia są skompilowane, zaczynamy więc właściwą część instalacji. Sprawdźmy instrukcję:

In [0]:
!cat kaldi/src/INSTALL

W pierwszej kolejności musimy wygenerować skrypty instalacyjne:

In [7]:
!cd kaldi/src && ./configure --shared

Configuring KALDI to use MKL.
Checking compiler g++ ...
Checking OpenFst library in /content/kaldi/tools/openfst-1.6.7 ...
Checking cub library in /content/kaldi/tools/cub-1.8.0 ...
Doing OS specific configurations ...
On Linux: Checking for linear algebra header files ...
Configuring MKL library directory: Found: /opt/intel/mkl/lib/intel64
MKL configured with threading: sequential, libs: -L/opt/intel/mkl/lib/intel64 -Wl,-rpath=/opt/intel/mkl/lib/intel64 -lmkl_intel_lp64  -lmkl_core  -lmkl_sequential 
MKL include directory configured as: /opt/intel/mkl/include
Configuring MKL threading as sequential
MKL threading libraries configured as   -ldl -lpthread -lm
Using Intel MKL as the linear algebra library.
Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
Successfully configured for Linux with MKL libs from /opt/intel/mkl
Using CUDA toolkit /usr/local/cuda (nvcc compiler and runtime libraries)
INFO: Configuring Kaldi not to link

Po wygenerowaniu skryptów budujących Kaldi, uruchamiamy je. Znów - chwilę to potrwa.

In [0]:
!cd kaldi/src && make depend -j2 && make -j2

Instalacja Kaldi powinna być teraz gotowa do użycia. Sprawdźmy w takim razie co znajduje się w pakiecie:

In [11]:
!tree -L 2 -d kaldi/

kaldi/
├── cmake
│   └── third_party
├── docker
│   ├── debian9.8-cpu
│   └── ubuntu16.04-gpu
├── egs
│   ├── aidatatang_200zh
│   ├── aishell
│   ├── aishell2
│   ├── ami
│   ├── an4
│   ├── apiai_decode
│   ├── aspire
│   ├── aurora4
│   ├── babel
│   ├── babel_multilang
│   ├── bentham
│   ├── bn_music_speech
│   ├── callhome_diarization
│   ├── callhome_egyptian
│   ├── casia_hwdb
│   ├── chime1
│   ├── chime2
│   ├── chime3
│   ├── chime4
│   ├── chime5
│   ├── chime6
│   ├── cifar
│   ├── cmu_cslu_kids
│   ├── cnceleb
│   ├── commonvoice
│   ├── csj
│   ├── dihard_2018
│   ├── fame
│   ├── farsdat
│   ├── fisher_callhome_spanish
│   ├── fisher_english
│   ├── fisher_swbd
│   ├── formosa
│   ├── gale_arabic
│   ├── gale_mandarin
│   ├── gop
│   ├── gp
│   ├── heroico
│   ├── hkust
│   ├── hub4_english
│   ├── hub4_spanish
│   ├── iam
│   ├── iban
│   ├── ifnenit
│   ├── librispeech
│   ├── lre
│   ├── lre07
│   ├── madcat_ar
│   ├── madcat_zh
│   ├── malach
│   ├── mandarin_bn_bc


- `kaldi/tools` - zewnętrzne toole niezbędne do działania Kaldi
- `kaldi/src` - “właściwy” kod Kaldi. Większość z programów ma jedną, dobrze określoną funkcję, co pozwala na łatwe składanie systemu ASR z różnych dostępnych klocków.
- `kaldi/egs` - “recepty trenujące” powiązane z konkretnymi zadaniami. Receptę uruchamia się plikiem `run.sh`.
- `kaldi/egs/$nazwa_recepty/$numer_recepty/steps` - uniwersalne skrypty wykonujące duże, konkretne zadania, np. zamianę plików wav na cechy MFCC, trening modelu monofonowego itp.
- `kaldi/egs/$nazwa_recepty/$numer_recepty/local` - skrypty i pliki przygotowane pod konkretną receptę (np. przetwarzające dane wejściowe)

Receptą którą wykorzystamy do wytrenowania prostego modelu rozpoznającego jezyk angielski jest recepta mini_librispeech. Jej zmodyfikowana pod Google Colab wersja, wraz z tymże notebookiem jest dostępna w repozytorium, które należy sklonować:

In [3]:
!git clone https://github.com/tvarog/kaldi-minilibrispeech-tutorial.git

Cloning into 'kaldi-minilibrispeech-tutorial'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 109 (delta 42), reused 101 (delta 37), pack-reused 0
Receiving objects: 100% (109/109), 63.58 KiB | 765.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


Następnie wkopiujmy receptę bezpośrednio do katalogu `kaldi/egs`:

In [4]:
cp -rvf kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/ kaldi/egs/

'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/' -> 'kaldi/egs/mini_librispeech_tutorial'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5' -> 'kaldi/egs/mini_librispeech_tutorial/s5'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/RESULTS' -> 'kaldi/egs/mini_librispeech_tutorial/s5/RESULTS'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/cmd.sh' -> 'kaldi/egs/mini_librispeech_tutorial/s5/cmd.sh'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/conf' -> 'kaldi/egs/mini_librispeech_tutorial/s5/conf'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/conf/decode.config' -> 'kaldi/egs/mini_librispeech_tutorial/s5/conf/decode.config'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/conf/mfcc.conf' -> 'kaldi/egs/mini_librispeech_tutorial/s5/conf/mfcc.conf'
'kaldi-minilibrispeech-tutorial/mini_librispeech_tutorial/s5/conf/mfcc_hires.conf' -> 'kaldi/egs/mini_librispeech_tutorial/s5/conf/mfcc_hires.conf'
'kaldi-

Jesteśmy gotowi do uruchomienia recepty. Receptę uruchamia się skryptem `run.sh`. Dla celów pokazowych została ona zmodyfikowana w taki sposób, żeby zatrzymać się po każdym wykonanym kroku. Konkretny krok wywołuje się parametrem `--stage $numer_kroku`.

Przyjrzyjmy się jak wygląda pierwszy krok:
```
if [ $stage -eq -1 ]; then
  for part in dev-clean-2 train-clean-5; do
    local/download_and_untar.sh $data $data_url $part
  done
fi
```
W skrócie - ściągamy i rozpakowujemy korpus mowy trenujący i deweloperski. Jest to tylko niewielki wycinek korpusu librispeech, okrojony dla celów demonstracyjnych.

In [5]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage -1

local/download_and_untar.sh: downloading data from www.openslr.org/resources/31/dev-clean-2.tar.gz.  This may take some time, please be patient.
--2020-05-14 21:45:31--  http://www.openslr.org/resources/31/dev-clean-2.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126046265 (120M) [application/x-gzip]
Saving to: 'dev-clean-2.tar.gz'

dev-clean-2.tar.gz  100%[===================>] 120.21M  29.0MB/s    in 4.1s    

2020-05-14 21:45:35 (29.0 MB/s) - 'dev-clean-2.tar.gz' saved [126046265/126046265]

/content/kaldi/egs/mini_librispeech_tutorial/s5
LibriSpeech/dev-clean-2/
LibriSpeech/dev-clean-2/5694/
LibriSpeech/dev-clean-2/5694/64038/
LibriSpeech/dev-clean-2/5694/64038/5694-64038.trans.txt
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0000.flac
LibriSpeech/dev-clean-2/5694/64038/5694-64038-0001.flac
LibriSpeech/dev-clean-2/5694/64038/5

Sprawdźmy co wypakowaliśmy:

In [23]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/corpus | head -n40

kaldi/egs/mini_librispeech_tutorial/s5/corpus
├── 3-gram.arpa.gz
├── 3-gram.pruned.1e-7.arpa.gz
├── 3-gram.pruned.3e-7.arpa.gz
├── dev-clean-2.tar.gz
├── LibriSpeech
│   ├── CHAPTERS.TXT
│   ├── dev-clean-2
│   │   ├── 1272
│   │   │   ├── 135031
│   │   │   │   ├── 1272-135031-0000.flac
│   │   │   │   ├── 1272-135031-0001.flac
│   │   │   │   ├── 1272-135031-0002.flac
│   │   │   │   ├── 1272-135031-0003.flac
│   │   │   │   ├── 1272-135031-0004.flac
│   │   │   │   ├── 1272-135031-0005.flac
│   │   │   │   ├── 1272-135031-0006.flac
│   │   │   │   ├── 1272-135031-0007.flac
│   │   │   │   ├── 1272-135031-0008.flac
│   │   │   │   ├── 1272-135031-0009.flac
│   │   │   │   ├── 1272-135031-0010.flac
│   │   │   │   ├── 1272-135031-0011.flac
│   │   │   │   ├── 1272-135031-0012.flac
│   │   │   │   ├── 1272-135031-0013.flac
│   │   │   │   ├── 1272-135031-0014.flac
│   │   │   │   ├── 1272-135031-0015.flac
│   │   │   │   ├── 1272-135031-0016.flac
│   │   │   │   ├── 1272-135031-0017.fl

W kolejnym kroku ściągniemy korpusy tekstowe:
```
if [ $stage -eq 0 ]; then
  local/download_lm.sh $lm_url $data data/local/lm
fi
```

In [14]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 0

--2020-05-14 22:31:41--  http://www.openslr.org/resources/11/3-gram.arpa.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759636181 (724M) [application/x-gzip]
Saving to: './corpus//3-gram.arpa.gz'

./corpus//3-gram.ar 100%[===================>] 724.45M  94.9MB/s    in 8.6s    

2020-05-14 22:31:50 (84.1 MB/s) - './corpus//3-gram.arpa.gz' saved [759636181/759636181]

--2020-05-14 22:31:50--  http://www.openslr.org/resources/11/3-gram.pruned.1e-7.arpa.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34094057 (33M) [application/x-gzip]
Saving to: './corpus//3-gram.pruned.1e-7.arpa.gz'

./corpus//3-gram.pr 100%[===================>]  32.51M  67.3MB/s    in 0.5s    

2020-05-14 22:31:51 (67.3 MB

Podejrzyjmy ściągnięte dane, najpierw model języka:

In [16]:
!zcat kaldi/egs/mini_librispeech_tutorial/s5/corpus//3-gram.arpa.gz | head -n40


\data\
ngram 1=200003
ngram 2=38229161
ngram 3=49712290

\1-grams:
-2.348754	</s>
-2.752519	<UNK>	-0.9697837
-99	<s>	-2.408548
-2.619969	A	-1.56262
-7.211563	A''S	-0.1495221
-6.221141	A'BODY	-0.2521624
-6.583487	A'COURT	-0.1844242
-6.240468	A'D	-0.2419162
-7.108924	A'GHA	-0.1740987
-6.260695	A'GOIN	-0.407374
-5.804425	A'LL	-0.3104859
-5.638036	A'M	-0.2983787
-6.221141	A'MIGHTY	-0.2186911
-6.885612	A'MIGHTY'S	-0.1800996
-4.996963	A'MOST	-0.4291549
-5.247992	A'N'T	-0.4541333
-6.68067	A'PENNY	-0.1663875
-5.518031	A'READY	-0.3329575
-6.202638	A'RIGHT	-0.3302211
-7.211563	A'RONY	-0.1495221
-5.439458	A'S	-0.277594
-5.618725	A'TER	-0.4115534
-6.504123	A'TERNOON	-0.2447588
-6.407031	A'TERWARDS	-0.2547638
-5.797244	A'THEGITHER	-0.2350724
-6.167841	A'THING	-0.2051675
-5.58687	A'TIM	-0.3298768
-5.929425	A'VE	-0.2925289
-5.489061	AA	-0.3251729
-6.407031	AAANTHOR	-0.2949072
-6.352588	AACHEN	-0.2925056
-6.352588	AAD	-0.1678229
-5.69044	AAGE	-0.3162718


Następnie prunowany (zmniejszony) model: 

In [19]:
!zcat kaldi/egs/mini_librispeech_tutorial/s5/corpus/3-gram.pruned.3e-7.arpa.gz | head -n40


\data\
ngram 1=200003
ngram 2=1016673
ngram 3=340026

\1-grams:
-2.348754	</s>
-2.752519	<UNK>	-0.1971074
-99	<s>	-0.9669631
-2.619969	A	-0.4922465
-7.211563	A''S
-6.221141	A'BODY	-0.07004431
-6.583487	A'COURT
-6.240468	A'D
-7.108924	A'GHA
-6.260695	A'GOIN	-0.2617707
-5.804425	A'LL	-0.06086903
-5.638036	A'M	-0.04902645
-6.221141	A'MIGHTY
-6.885612	A'MIGHTY'S
-4.996963	A'MOST	-0.1447615
-5.247992	A'N'T	-0.1053417
-6.68067	A'PENNY
-5.518031	A'READY	-0.1232338
-6.202638	A'RIGHT	-0.1630445
-7.211563	A'RONY
-5.439458	A'S	-0.0566647
-5.618725	A'TER	-0.1851806
-6.504123	A'TERNOON
-6.407031	A'TERWARDS
-5.797244	A'THEGITHER	-0.007919221
-6.167841	A'THING
-5.58687	A'TIM	-0.1079524
-5.929425	A'VE
-5.489061	AA	-0.09303241
-6.407031	AAANTHOR	-0.1265862
-6.352588	AACHEN
-6.352588	AAD
-5.69044	AAGE


Następnie leksykon:

In [17]:
!head -n40 kaldi/egs/mini_librispeech_tutorial/s5/corpus/librispeech-lexicon.txt

A  AH0
A  EY1
A''S	EY1 Z
A'BODY	EY1 B AA2 D IY0
A'COURT	EY1 K AO2 R T
A'D	EY1 D
A'GHA	EY1 G AH0
A'GOIN	EY1 G OY1 N
A'LL	EY1 L
A'M	EY1 M
A'MIGHTY	EY1 M AY1 T IY0
A'MIGHTY'S	EY1 M AY1 T IY0 Z
A'MOST	EY1 M OW2 S T
A'N'T	EY1 AH0 N T
A'PENNY	EY1 P EH2 N IY0
A'READY	EY1 R IY1 D IY0
A'RIGHT	EY1 R AY2 T
A'RONY	EY1 R OW1 N IY0
A'S  EY1 Z
A'TER	EY1 T ER0
A'TERNOON	EY1 T ER0 N UW1 N
A'TERWARDS	EY1 T ER0 W ER0 D Z
A'THEGITHER	EY1 DH AH0 JH IH1 DH ER0
A'THING	EY1 DH IH0 NG
A'TIM	EY1 T IH2 M
A'VE	AH0 V
AA	AA1
AAANTHOR	T R IH2 P AH0 L EY1 N TH ER0
AACHEN  AA1 K AH0 N
AAD	AA1 D
AAGE	AA1 ZH
AAGE'S	IH0 JH IH0 Z
AAGOT	AA1 G AH0 T
AAGOT'S	AA1 G AH0 T S
AAH	AA1
AAHMES	AA1 M Z
AAKRE	AA1 K ER0
AAL	AA1 L
AALBOM	AA1 L B AO2 M
AALST	AA1 L S T


W kolejnym kroku bierzemy surowe, ściągnięte dane i przygotowujemy je do wykorzystania w Kaldi:
```
if [ $stage -eq 1 ]; then
  # format the data as Kaldi data directories
  for part in dev-clean-2 train-clean-5; do
    # use underscore-separated names in data directories.
    local/data_prep.sh $data/LibriSpeech/$part data/$(echo $part | sed s/-/_/g)
  done

  # take only 100 first samples from dev_clean_2 to speed-up model evaluation
  cp -rf data/dev_clean_2 data/dev_clean_2.bak
  head -n100 data/dev_clean_2.bak/wav.scp > data/dev_clean_2/wav.scp
  ./utils/fix_data_dir.sh data/dev_clean_2

  local/prepare_dict.sh --stage 3 --nj $nj --cmd "$train_cmd" \
    data/local/lm data/local/lm data/local/dict_nosp

  utils/prepare_lang.sh data/local/dict_nosp \
    "<UNK>" data/local/lang_tmp_nosp data/lang_nosp

  local/format_lms.sh --src-dir data/lang_nosp data/local/lm
  # Create ConstArpaLm format language model for full 3-gram and 4-gram LMs
  utils/build_const_arpa_lm.sh data/local/lm/lm_tgmed.arpa.gz \
    data/lang_nosp data/lang_nosp_test_tgmed
fi
```

In [24]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 1

utils/validate_data_dir.sh: Successfully validated data-directory data/dev_clean_2
local/data_prep.sh: successfully prepared data in data/dev_clean_2
utils/validate_data_dir.sh: Successfully validated data-directory data/train_clean_5
local/data_prep.sh: successfully prepared data in data/train_clean_5
fix_data_dir.sh: kept 100 utterances out of 1089
./utils/fix_data_dir.sh: filtered data/dev_clean_2/spk2gender from 38 to 3 lines based on filter /tmp/kaldi.kYSP/speakers.
fix_data_dir.sh: old files are kept in data/dev_clean_2/.backup
Preparing phone lists and clustering questions
2 silence phones saved to: data/local/dict_nosp/silence_phones.txt
1 optional silence saved to: data/local/dict_nosp/optional_silence.txt
39 non-silence phones saved to: data/local/dict_nosp/nonsilence_phones.txt
5 extra triphone clustering-related questions saved to: data/local/dict_nosp/extra_questions.txt
Lexicon text file saved as: data/local/dict_nosp/lexicon.txt
utils/prepare_lang.sh data/local/dict_nosp

Finalnie przygotowane dane znajdują się w katalogu `data`, sprawdźmy co leży w środku:

In [25]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/data

kaldi/egs/mini_librispeech_tutorial/s5/data
├── dev_clean_2
│   ├── spk2gender
│   ├── spk2utt
│   ├── text
│   ├── utt2spk
│   └── wav.scp
├── dev_clean_2.bak
│   ├── spk2gender
│   ├── spk2utt
│   ├── text
│   ├── utt2spk
│   └── wav.scp
├── lang_nosp
│   ├── L_disambig.fst
│   ├── L.fst
│   ├── oov.int
│   ├── oov.txt
│   ├── phones
│   │   ├── align_lexicon.int
│   │   ├── align_lexicon.txt
│   │   ├── context_indep.csl
│   │   ├── context_indep.int
│   │   ├── context_indep.txt
│   │   ├── disambig.csl
│   │   ├── disambig.int
│   │   ├── disambig.txt
│   │   ├── extra_questions.int
│   │   ├── extra_questions.txt
│   │   ├── nonsilence.csl
│   │   ├── nonsilence.int
│   │   ├── nonsilence.txt
│   │   ├── optional_silence.csl
│   │   ├── optional_silence.int
│   │   ├── optional_silence.txt
│   │   ├── roots.int
│   │   ├── roots.txt
│   │   ├── sets.int
│   │   ├── sets.txt
│   │   ├── silence.csl
│   │   ├── silence.int
│   │   ├── silence.txt
│   │   ├── wdisambig_phones.int
│ 

Sprawdźmy najpierw korpus trenujący. W pliku `text` znajdują się tzw. referencje, czyli transkrypcje mowy:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/text | head -n20

W pliku `wav.scp` znajdują się ścieżki do plików audio:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/wav.scp | head -n20

W pliku `spk2utt` znajdują się mappingi speaker -> ID pliku 

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/spk2utt | head -n20

Odwrotny mapping znajduje się w pliku `utt2spk`:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/utt2spk | head -n20

Mapping speaker -> płeć znajduje się w `spk2gender`:

In [0]:
cat kaldi/egs/mini_librispeech_tutorial/s5/data/train_clean_5/spk2gender | head -n20

Dane fonetyczne znajdują się z kolei w `data/local/dict_nosp`:

In [0]:
!head -n20 kaldi/egs/mini_librispeech_tutorial/s5/data/local/dict_nosp/*

Kolejnym krokiem jest zamiana plików audio na cechy sygnału:
```
if [ $stage -eq 2 ]; then
  mfccdir=mfcc

  for part in dev_clean_2 train_clean_5; do
    steps/make_mfcc.sh --cmd "$train_cmd" --nj $nj data/$part exp/make_mfcc/$part $mfccdir
    steps/compute_cmvn_stats.sh data/$part exp/make_mfcc/$part $mfccdir
  done

  # Get the shortest 500 utterances first because those are more likely
  # to have accurate alignments.
  utils/subset_data_dir.sh --shortest data/train_clean_5 500 data/train_500short
fi
```

In [63]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 2

steps/make_mfcc.sh --cmd run.pl --mem 2G --nj 2 data/dev_clean_2 exp/make_mfcc/dev_clean_2 mfcc
steps/make_mfcc.sh: moving data/dev_clean_2/feats.scp to data/dev_clean_2/.backup
utils/validate_data_dir.sh: Successfully validated data-directory data/dev_clean_2
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for dev_clean_2
steps/compute_cmvn_stats.sh data/dev_clean_2 exp/make_mfcc/dev_clean_2 mfcc
Succeeded creating CMVN stats for dev_clean_2
steps/make_mfcc.sh --cmd run.pl --mem 2G --nj 2 data/train_clean_5 exp/make_mfcc/train_clean_5 mfcc
steps/make_mfcc.sh: moving data/train_clean_5/feats.scp to data/train_clean_5/.backup
utils/validate_data_dir.sh: Successfully validated data-directory data/train_clean_5
steps/make_mfcc.sh: [info]: no segments file exists: assuming wav.scp indexed by utterance.
steps/make_mfcc.sh: Succeeded creating MFCC features for train_clean_5
steps/compute_cmvn_sta

Wygenerowane cechy MFCC znajdują się w katalogu `mfcc`, podejrzyjmy też logi w `exp/make_mfcc/` 

In [0]:
!ls -l kaldi/egs/mini_librispeech_tutorial/s5/mfcc
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/make_mfcc/dev_clean_2/make_mfcc_dev_clean_2.1.log

Mamy gotowe już wszystkie niezbędne do wytrenowania systemu dane:
- cechy sygnału (czyli wejście do modelu akustycznego)
- słownik fonetyczny (czyli połączenie wejścia do modelu językowego i wyjscia z modelu akustycznego)
- model języka

Zacznijmy od wytrenowania modelu monofonowego:
```
if [ $stage -eq 3 ]; then
  steps/train_mono.sh --boost-silence 1.25 --nj $nj --cmd "$train_cmd" \
    data/train_500short data/lang_nosp exp/mono
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/mono exp/mono/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/mono/graph_nosp_tgsmall \
        data/$test exp/mono/decode_nosp_tgsmall_$test
    done
  )

  steps/align_si.sh --boost-silence 1.25 --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/mono exp/mono_ali_train_clean_5
fi
```

In [38]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 3

steps/train_mono.sh --boost-silence 1.25 --nj 2 --cmd run.pl --mem 2G data/train_500short data/lang_nosp exp/mono
steps/train_mono.sh: Initializing monophone system.
steps/train_mono.sh: Compiling training graphs
steps/train_mono.sh: Aligning data equally (pass 0)
steps/train_mono.sh: Pass 1
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 2
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 3
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 4
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 5
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 6
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 7
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 8
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 9
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 10
steps/train_mono.sh: Aligning data
steps/train_mono.sh: Pass 11
steps/train_mono.sh: Pass 12
steps/train_mono.sh: Aligning data
steps/train

Sprawdźmy output z treningu:

In [39]:
!tree kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/

kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/
├── 0.mdl
├── 40.mdl
├── 40.occs
├── ali.1.gz
├── ali.2.gz
├── cmvn_opts
├── decode_nosp_tgsmall_dev_clean_2
│   ├── lat.1.gz
│   ├── lat.2.gz
│   ├── log
│   │   ├── analyze_alignments.log
│   │   ├── analyze_lattice_depth_stats.log
│   │   ├── decode.1.log
│   │   ├── decode.2.log
│   │   ├── get_lattice_stats.1.log
│   │   ├── get_lattice_stats.2.log
│   │   ├── lattice_best_path.1.log
│   │   └── lattice_best_path.2.log
│   ├── num_jobs
│   ├── scoring
│   │   ├── 10.0.0.tra
│   │   ├── 10.0.5.tra
│   │   ├── 10.1.0.tra
│   │   ├── 11.0.0.tra
│   │   ├── 11.0.5.tra
│   │   ├── 11.1.0.tra
│   │   ├── 12.0.0.tra
│   │   ├── 12.0.5.tra
│   │   ├── 12.1.0.tra
│   │   ├── 13.0.0.tra
│   │   ├── 13.0.5.tra
│   │   ├── 13.1.0.tra
│   │   ├── 14.0.0.tra
│   │   ├── 14.0.5.tra
│   │   ├── 14.1.0.tra
│   │   ├── 15.0.0.tra
│   │   ├── 15.0.5.tra
│   │   ├── 15.1.0.tra
│   │   ├── 16.0.0.tra
│   │   ├── 16.0.5.tra
│   │   ├── 16.1.0.tra
│   │  

Interesuje nas to jak radzi sobie wytrenowany model w praktyce. Zerknijmy na logi z dekodera i porównajmy wynik z referencjami z `data/dev_clean_2/text`


In [43]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log

1272-135031-0000 BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
1272-135031-0001 HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
1272-135031-0002 I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
1272-135031-0003 THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD THE RAPS AND OPENED THE DOOR
1272-135031-0004 THE KING HAS FLED IN DISGRACE AND YOUR FRIENDS ARE ASKING FOR YOU
# gmm-latgen-faster --max-active=7000 --beam=13.0 --lattice-beam=6.0 --acoustic-scale=0.083333 --allow-partial=true --word-symbol-table=exp/mono/graph_nosp_tgsmall/words.txt exp/mono/final.mdl exp/mono/graph_nosp_tgsmall/HCLG.fst "ark,s,cs:apply-cmvn  --utt2spk=ark:data/dev_clean_2/split2/1/utt2spk scp:d

Sprawdźmy też ile wynosi Word Error Rate:

In [44]:
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/mono/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0

compute-wer --text --mode=present ark:exp/mono/decode_nosp_tgsmall_dev_clean_2/scoring/test_filt.txt ark,p:- 
%WER 56.46 [ 774 / 1371, 24 ins, 211 del, 539 sub ]
%SER 93.00 [ 93 / 100 ]
Scored 100 sentences, 0 not present in hyp.


Cóż, rozpoznajemy dobrze co drugie słowo. Nie najlepiej, ale pamiętajmy, że wytrenowaliśmy bardzo prosty model korzystając z bardzo małej ilości danych.

Korzystając z modelu monofonowego, wytrenujmy lepszy model - model trifonowy:
```
if [ $stage -eq 4 ]; then
  steps/train_deltas.sh --boost-silence 1.25 --cmd "$train_cmd" \
    2000 10000 data/train_clean_5 data/lang_nosp exp/mono_ali_train_clean_5 exp/tri1

  # decode using the tri1 model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri1 exp/tri1/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/tri1/graph_nosp_tgsmall \
      data/$test exp/tri1/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri1/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri1/decode_nosp_{tgsmall,tgmed}_$test
    done
  )

  steps/align_si.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri1 exp/tri1_ali_train_clean_5
fi
```

In [45]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 4

steps/train_deltas.sh --boost-silence 1.25 --cmd run.pl --mem 2G 2000 10000 data/train_clean_5 data/lang_nosp exp/mono_ali_train_clean_5 exp/tri1
steps/train_deltas.sh: accumulating tree stats
steps/train_deltas.sh: getting questions for tree-building, via clustering
steps/train_deltas.sh: building the tree
steps/train_deltas.sh: converting alignments from exp/mono_ali_train_clean_5 to use current tree
steps/train_deltas.sh: compiling graphs of transcripts
steps/train_deltas.sh: training pass 1
steps/train_deltas.sh: training pass 2
steps/train_deltas.sh: training pass 3
steps/train_deltas.sh: training pass 4
steps/train_deltas.sh: training pass 5
steps/train_deltas.sh: training pass 6
steps/train_deltas.sh: training pass 7
steps/train_deltas.sh: training pass 8
steps/train_deltas.sh: training pass 9
steps/train_deltas.sh: training pass 10
steps/train_deltas.sh: aligning data
steps/train_deltas.sh: training pass 11
steps/train_deltas.sh: training pass 12
steps/train_deltas.sh: training

Tym razem sprawdzamy wytrenowany model akustyczny korzystając z małego i średniego modelu języka (tgsmall, tgmed):

In [46]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri1/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

1272-135031-0000 BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
1272-135031-0001 HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
1272-135031-0002 I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
1272-135031-0003 THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD THE RAPS AND OPENED THE DOOR
1272-135031-0004 THE KING HAS FLED IN DISGRACE AND YOUR FRIENDS ARE ASKING FOR YOU
# gmm-latgen-faster --max-active=7000 --beam=13.0 --lattice-beam=6.0 --acoustic-scale=0.083333 --allow-partial=true --word-symbol-table=exp/tri1/graph_nosp_tgsmall/words.txt exp/tri1/final.mdl exp/tri1/graph_nosp_tgsmall/HCLG.fst "ark,s,cs:apply-cmvn  --utt2spk=ark:data/dev_clean_2/split2/1/utt2spk scp:d

Jest dużo lepiej, ale wciąż kiepsko.

Wytrenujmy kolejny model, model trifonowy z cechami zdekorelowanymi przy użyciu LDA:
```
if [ $stage -eq 5 ]; then
  steps/train_lda_mllt.sh --cmd "$train_cmd" \
    --splice-opts "--left-context=3 --right-context=3" 2500 15000 \
    data/train_clean_5 data/lang_nosp exp/tri1_ali_train_clean_5 exp/tri2b

  # decode using the LDA+MLLT model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri2b exp/tri2b/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode.sh --nj $nj --cmd "$decode_cmd" exp/tri2b/graph_nosp_tgsmall \
        data/$test exp/tri2b/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri2b/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri2b/decode_nosp_{tgsmall,tgmed}_$test
    done
  )

  # Align utts using the tri2b model
  steps/align_si.sh  --nj $nj --cmd "$train_cmd" --use-graphs true \
    data/train_clean_5 data/lang_nosp exp/tri2b exp/tri2b_ali_train_clean_5
fi
``` 

In [47]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 5

steps/train_lda_mllt.sh --cmd run.pl --mem 2G --splice-opts --left-context=3 --right-context=3 2500 15000 data/train_clean_5 data/lang_nosp exp/tri1_ali_train_clean_5 exp/tri2b
steps/train_lda_mllt.sh: Accumulating LDA statistics.
steps/train_lda_mllt.sh: Accumulating tree stats
steps/train_lda_mllt.sh: Getting questions for tree clustering.
steps/train_lda_mllt.sh: Building the tree
steps/train_lda_mllt.sh: Initializing the model
steps/train_lda_mllt.sh: Converting alignments from exp/tri1_ali_train_clean_5 to use current tree
steps/train_lda_mllt.sh: Compiling graphs of transcripts
Training pass 1
Training pass 2
steps/train_lda_mllt.sh: Estimating MLLT
Training pass 3
Training pass 4
steps/train_lda_mllt.sh: Estimating MLLT
Training pass 5
Training pass 6
steps/train_lda_mllt.sh: Estimating MLLT
Training pass 7
Training pass 8
Training pass 9
Training pass 10
Aligning data
Training pass 11
Training pass 12
steps/train_lda_mllt.sh: Estimating MLLT
Training pass 13
Training pass 14
Tr

Sprawdźmy wyniki:

In [48]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri2b/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

1272-135031-0000 BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
1272-135031-0001 HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
1272-135031-0002 I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
1272-135031-0003 THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD THE RAPS AND OPENED THE DOOR
1272-135031-0004 THE KING HAS FLED IN DISGRACE AND YOUR FRIENDS ARE ASKING FOR YOU
# gmm-latgen-faster --max-active=7000 --beam=13.0 --lattice-beam=6.0 --acoustic-scale=0.083333 --allow-partial=true --word-symbol-table=exp/tri2b/graph_nosp_tgsmall/words.txt exp/tri2b/final.mdl exp/tri2b/graph_nosp_tgsmall/HCLG.fst "ark,s,cs:apply-cmvn  --utt2spk=ark:data/dev_clean_2/split2/1/utt2spk sc

Znów poprawa.

Dodajmy dodatkową adaptację modelu per speaker (SAT):
```
if [ $stage -eq 6 ]; then
  steps/train_sat.sh --cmd "$train_cmd" 2500 15000 \
    data/train_clean_5 data/lang_nosp exp/tri2b_ali_train_clean_5 exp/tri3b

  # decode using the tri3b model
  (
    utils/mkgraph.sh data/lang_nosp_test_tgsmall \
      exp/tri3b exp/tri3b/graph_nosp_tgsmall
    for test in dev_clean_2; do
      steps/decode_fmllr.sh --nj $nj --cmd "$decode_cmd" \
        exp/tri3b/graph_nosp_tgsmall data/$test \
        exp/tri3b/decode_nosp_tgsmall_$test
      steps/lmrescore.sh --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri3b/decode_nosp_{tgsmall,tgmed}_$test
      steps/lmrescore_const_arpa.sh \
        --cmd "$decode_cmd" data/lang_nosp_test_{tgsmall,tgmed} \
        data/$test exp/tri3b/decode_nosp_{tgsmall,tgmed}_$test
    done
  )
fi
```

In [49]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 6

steps/train_sat.sh --cmd run.pl --mem 2G 2500 15000 data/train_clean_5 data/lang_nosp exp/tri2b_ali_train_clean_5 exp/tri3b
steps/train_sat.sh: feature type is lda
steps/train_sat.sh: obtaining initial fMLLR transforms since not present in exp/tri2b_ali_train_clean_5
steps/train_sat.sh: Accumulating tree stats
steps/train_sat.sh: Getting questions for tree clustering.
steps/train_sat.sh: Building the tree
steps/train_sat.sh: Initializing the model
steps/train_sat.sh: Converting alignments from exp/tri2b_ali_train_clean_5 to use current tree
steps/train_sat.sh: Compiling graphs of transcripts
Pass 1
Pass 2
Estimating fMLLR transforms
Pass 3
Pass 4
Estimating fMLLR transforms
Pass 5
Pass 6
Estimating fMLLR transforms
Pass 7
Pass 8
Pass 9
Pass 10
Aligning data
Pass 11
Pass 12
Estimating fMLLR transforms
Pass 13
Pass 14
Pass 15
Pass 16
Pass 17
Pass 18
Pass 19
Pass 20
Aligning data
Pass 21
Pass 22
Pass 23
Pass 24
Pass 25
Pass 26
Pass 27
Pass 28
Pass 29
Pass 30
Aligning data
Pass 31
Pass 32


Sprawdźmy wyniki:

In [50]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_nosp_tgmed_dev_clean_2/wer_10_0.0

1272-135031-0000 BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
1272-135031-0001 HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
1272-135031-0002 I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
1272-135031-0003 THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD THE RAPS AND OPENED THE DOOR
1272-135031-0004 THE KING HAS FLED IN DISGRACE AND YOUR FRIENDS ARE ASKING FOR YOU
# gmm-latgen-faster --max-active=7000 --beam=13.0 --lattice-beam=6.0 --acoustic-scale=0.083333 --determinize-lattice=false --allow-partial=true --word-symbol-table=exp/tri3b/graph_nosp_tgsmall/words.txt exp/tri3b/final.mdl exp/tri3b/graph_nosp_tgsmall/HCLG.fst "ark,s,cs:apply-cmvn  --utt2spk=ark:data/dev

Wyniki zaczynają być niezłe.

Do tej pory zakładaliśmy, że różne wymowy tych samych słów mają identyczne prawdopodobieństwo wystąpienia. Nie jest to prawda. W poniższym kroku sprawdzimy jaki jest rozkład różnych wymów w korpusie trenującym.
```
if [ $stage -eq 7 ]; then
  steps/get_prons.sh --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri3b
  utils/dict_dir_add_pronprobs.sh --max-normalize true \
    data/local/dict_nosp \
    exp/tri3b/pron_counts_nowb.txt exp/tri3b/sil_counts_nowb.txt \
    exp/tri3b/pron_bigram_counts_nowb.txt data/local/dict

  utils/prepare_lang.sh data/local/dict \
    "<UNK>" data/local/lang_tmp data/lang

  local/format_lms.sh --src-dir data/lang data/local/lm

  utils/build_const_arpa_lm.sh \
    data/local/lm/lm_tgmed.arpa.gz data/lang data/lang_test_tgmed

  steps/align_fmllr.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang exp/tri3b exp/tri3b_ali_train_clean_5
fi
```

In [51]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 7

steps/get_prons.sh --cmd run.pl --mem 2G data/train_clean_5 data/lang_nosp exp/tri3b
steps/get_prons.sh: exp/tri3b/ali.1.gz exists, so starting from alignments.
steps/get_prons.sh: done writing prons to exp/tri3b/prons.*.gz, silence counts in 
steps/get_prons.sh: exp/tri3b/sil_counts_nowb.txt and pronunciation counts in 
steps/get_prons.sh: exp/tri3b/pron_counts.{int,txt}
steps/get_prons.sh: ... and also in exp/tri3b/pron_counts_nowb.txt
Checking data/local/dict_nosp/silence_phones.txt ...
--> reading data/local/dict_nosp/silence_phones.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict_nosp/silence_phones.txt is OK

Checking data/local/dict_nosp/optional_silence.txt ...
--> reading data/local/dict_nosp/optional_silence.txt
--> text seems to be UTF-8 or ASCII, checking whitespaces
--> text contains only allowed whitespaces
--> data/local/dict_nosp/optional_silence.txt is OK

Checking data/local/dict_nosp/nonsilen

Zerknijmy do zaktualizowanego słownika:

In [61]:
!grep "^WITH " kaldi/egs/mini_librispeech_tutorial/s5/data/local/dict/lexiconp.txt

WITH 0.0600001 W IH1 DH
WITH 0.296 W IH1 TH
WITH 0.348 W IH0 DH
WITH 1 W IH0 TH


W następnym kroku wykorzystamy nowe informacje o słowniku i ponownie wytrenujemy ostatni model (SAT):
```
if [ $stage -eq 7 ]; then
  steps/get_prons.sh --cmd "$train_cmd" \
    data/train_clean_5 data/lang_nosp exp/tri3b
  utils/dict_dir_add_pronprobs.sh --max-normalize true \
    data/local/dict_nosp \
    exp/tri3b/pron_counts_nowb.txt exp/tri3b/sil_counts_nowb.txt \
    exp/tri3b/pron_bigram_counts_nowb.txt data/local/dict

  utils/prepare_lang.sh data/local/dict \
    "<UNK>" data/local/lang_tmp data/lang

  local/format_lms.sh --src-dir data/lang data/local/lm

  utils/build_const_arpa_lm.sh \
    data/local/lm/lm_tgmed.arpa.gz data/lang data/lang_test_tgmed

  steps/align_fmllr.sh --nj $nj --cmd "$train_cmd" \
    data/train_clean_5 data/lang exp/tri3b exp/tri3b_ali_train_clean_5
fi
```

In [62]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 8

tree-info exp/tri3b/tree 
tree-info exp/tri3b/tree 
fstpushspecial 
fsttablecompose data/lang_test_tgsmall/L_disambig.fst data/lang_test_tgsmall/G.fst 
fstdeterminizestar --use-log=true 
fstminimizeencoded 
fstisstochastic data/lang_test_tgsmall/tmp/LG.fst 
-0.0383488 -0.0392972
[info]: LG not stochastic.
fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=data/lang_test_tgsmall/phones/disambig.int --write-disambig-syms=data/lang_test_tgsmall/tmp/disambig_ilabels_3_1.int data/lang_test_tgsmall/tmp/ilabels_3_1.43090 data/lang_test_tgsmall/tmp/LG.fst 
fstisstochastic data/lang_test_tgsmall/tmp/CLG_3_1.fst 
0 -0.0392972
[info]: CLG not stochastic.
make-h-transducer --disambig-syms-out=exp/tri3b/graph_tgsmall/disambig_tid.int --transition-scale=1.0 data/lang_test_tgsmall/tmp/ilabels_3_1 exp/tri3b/tree exp/tri3b/final.mdl 
fstrmepslocal 
fstminimizeencoded 
fstdeterminizestar --use-log=true 
fsttablecompose exp/tri3b/graph_tgsmall/Ha.fst data/lang_test_tgsmall/tmp/C

Sprawdźmy wyniki:

In [65]:
!head -n5 kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/text
!head -n15 kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgsmall_dev_clean_2/log/decode.1.log
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/tri3b/decode_tgmed_dev_clean_2/wer_10_0.0

1272-135031-0000 BECAUSE YOU WERE SLEEPING INSTEAD OF CONQUERING THE LOVELY ROSE PRINCESS HAS BECOME A FIDDLE WITHOUT A BOW WHILE POOR SHAGGY SITS THERE A COOING DOVE
1272-135031-0001 HE HAS GONE AND GONE FOR GOOD ANSWERED POLYCHROME WHO HAD MANAGED TO SQUEEZE INTO THE ROOM BESIDE THE DRAGON AND HAD WITNESSED THE OCCURRENCES WITH MUCH INTEREST
1272-135031-0002 I HAVE REMAINED A PRISONER ONLY BECAUSE I WISHED TO BE ONE AND WITH THIS HE STEPPED FORWARD AND BURST THE STOUT CHAINS AS EASILY AS IF THEY HAD BEEN THREADS
1272-135031-0003 THE LITTLE GIRL HAD BEEN ASLEEP BUT SHE HEARD THE RAPS AND OPENED THE DOOR
1272-135031-0004 THE KING HAS FLED IN DISGRACE AND YOUR FRIENDS ARE ASKING FOR YOU
# gmm-latgen-faster --max-active=7000 --beam=13.0 --lattice-beam=6.0 --acoustic-scale=0.083333 --determinize-lattice=false --allow-partial=true --word-symbol-table=exp/tri3b/graph_tgsmall/words.txt exp/tri3b/final.mdl exp/tri3b/graph_tgsmall/HCLG.fst "ark,s,cs:apply-cmvn  --utt2spk=ark:data/dev_clean_2/s

Jest już naprawdę dobrze (biorąc pod uwagę mikroskopijny zbiór uczący). Jeśli chcemy wytrenować lepszy model pozostają nam sieci neuronowe:
```
# Train a chain model
if [ $stage -eq 9 ]; then
  local/chain/run_tdnn.sh --stage 0
fi
```
Dla zainteresowanych, opis sieci TDNN wykorzystanej w Kaldi: https://en.wikipedia.org/wiki/Time_delay_neural_network

In [0]:
!cd kaldi/egs/mini_librispeech_tutorial/s5/ && bash run.sh --stage 9

Nie pozostaje nam nic innego niż obejrzeć wyniki:

In [0]:
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp/decode_tgmed_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp_online/decode_tgsmall_dev_clean_2/wer_10_0.0
!cat kaldi/egs/mini_librispeech_tutorial/s5/exp/chain/tdnn1h_sp_online/decode_tgmed_dev_clean_2/wer_10_0.0

Spadek liczby błędów jest znaczący, daleko nam jednak do jakości oferowanej przez komercyjne systemy ASR.

---

###Zadanie 1
1. Znajdź nagranie z angielską mową.
1. Skonwertuj nagranie do pliku wav, 16kHz, mono (np. w Audacity).
1. Zuploaduj nagranie na instancję.
1. Podmień `flac.*$` w ostatniej linijce `kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/wav.scp` na ścieżkę do zuploadowanego pliku. Tip: żeby otrzymać pełną ścieżkę do pliku użyj polecenia `readlink -f PLIK`
1. Podmień tekst w ostatniej linijce `kaldi/egs/mini_librispeech_tutorial/s5/data/dev_clean_2/wav.scp` na faktyczną zawartość nowego pliku
1. Przegeneruj cechy sygnału - MFCC (stage 2)
1. Uruchom dowolny trening (np. SAT) i sprawdź logach z dekodera co zostało rozpoznane. Wynik będzie prawdopodobnie na końcu logu `decode.2.log`
1. Jako zaliczenie, wyślij plik `decode.2.log`

###Zadanie 2
1. Przygotuj maszynę wirtualną z Ubuntu 18.04.
1. Przygotuj środowisko i skompiluj Kaldi, tak jak w omawianym dziś przykładzie.
1. Wytrenuj model Kaldi na dużym zbiorze trenującym.
1. Skorzystaj z recepty kaldi/egs/librispeech/s5.
1. Jako zaliczenie, wyślij pliki raport porównujący wyniki na minilibrispeech i librispeech.

Kilka ważnych uwag:
- w pliku `cmd.sh` należy zmienić zawartość wszystkich zmiennych na `run.pl`
- w pliku run.sh należy zmienić liczbę jobów przy każdym parametrze `--nj LICZBA` na liczbę rdzeni dostępnych na maszynie wirtualnej. Uwaga na pamięć!
- w pliku run.sh należy zmienić wszystkie wystąpienia `tglarge` na `tgmed`. Na zwykłych PC model językowy `tglarge` może być zbyt duży do skompilowania.
- tym razem skrypt run.sh nie zatrzymuje się po każdym kroku. Parametr `--stage LICZBA` kontroluje teraz tylko startowy krok.
- skończ na stage 13. 